In [1]:
!git clone https://github.com/isayev/ReLeaSE.git

Cloning into 'ReLeaSE'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 548 (delta 0), reused 2 (delta 0), pack-reused 545
Receiving objects: 100% (548/548), 449.76 MiB | 8.72 MiB/s, done.
Resolving deltas: 100% (264/264), done.


In [2]:
!pip install rdkit mordred

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [51]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

In [4]:
sys.path.append('./release/')
%cd ReLeaSE/release

/home/sl-beast/manikanta/ReLeaSE/release


In [5]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [6]:
use_cuda = torch.cuda.is_available()

In [7]:
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
from stackRNN import StackAugmentedRNN
from data import GeneratorData
from utils import canonical_smiles

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [17]:
gen_data_path = '/home/sl-beast/manikanta/data_train.txt'

In [18]:
tokens = ['<', '>', '#', '%', ')', '(', '+', '-', '/', '.', '1', '0', '3', '2', '5', '4', '7',
          '6', '9', '8', '=', 'A', '@', 'C', 'B', 'F', 'I', 'H', 'O', 'N', 'P', 'S', '[', ']',
          '\\', 'c', 'a', 'e', 'i', 'l', 'o', 'n', 'p', 's', 'r', '\n']

In [19]:
gen_data = GeneratorData(training_data_path=gen_data_path, delimiter=' ',
                         cols_to_read=[0], keep_header=True, tokens=tokens)

In [20]:
def plot_hist(prediction, n_to_generate):
    prediction = np.array(prediction)
    percentage_in_threshold = np.sum((prediction >= 0.0) &
                                     (prediction <= 5.0))/len(prediction)
    print("Percentage of predictions within drug-like region:", percentage_in_threshold)
    print("Proportion of valid SMILES:", len(prediction)/n_to_generate)
    ax = sns.kdeplot(prediction, shade=True)
    plt.axvline(x=0.0)
    plt.axvline(x=5.0)
    ax.set(xlabel='Predicted LogP',
           title='Distribution of predicted LogP for generated molecules')
    plt.show()

In [21]:
def estimate_and_update(generator, predictor, n_to_generate):
    generated = []
    pbar = tqdm(range(n_to_generate))
    for i in pbar:
        pbar.set_description("Generating molecules...")
        generated.append(generator.evaluate(gen_data, predict_len=120)[1:-1])

    sanitized = canonical_smiles(generated, sanitize=False, throw_warning=False)[:-1]
    unique_smiles = list(np.unique(sanitized))[1:]
    smiles, prediction, nan_smiles = predictor.predict(unique_smiles, use_tqdm=True)

    plot_hist(prediction, n_to_generate)

    return smiles, prediction

In [22]:
hidden_size = 1500
stack_width = 1500
stack_depth = 200
layer_type = 'GRU'
lr = 0.001
optimizer_instance = torch.optim.Adadelta

my_generator = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                 output_size=gen_data.n_characters, layer_type=layer_type,
                                 n_layers=1, is_bidirectional=False, has_stack=True,
                                 stack_width=stack_width, stack_depth=stack_depth,
                                 use_cuda=use_cuda,
                                 optimizer_instance=optimizer_instance, lr=lr)

In [23]:
#losses = my_generator.fit(gen_data, 1500000)

In [24]:
# i = 0
# while i<100001:
#   if i % 10000 == 0:
#     my_generator.save_model(f'/content/{i}')
#   my_generator.load_model(f'/content/drive/MyDrive/colab/{i}')
#   losses = my_generator.fit(gen_data, 10000)
#   i += 10000
# my_generator.save_model(f'/content/{i}')

In [19]:
#my_generator.save_model('/content/20000')

In [25]:
my_generator.load_model('/home/sl-beast/manikanta/410000')

In [26]:
! git clone https://github.com/Mariewelt/OpenChem.git

Cloning into 'OpenChem'...
remote: Enumerating objects: 2179, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 2179 (delta 62), reused 126 (delta 61), pack-reused 2050
Receiving objects: 100% (2179/2179), 166.23 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


In [27]:
from rnn_predictor import RNNPredictor

2023-12-06 15:54:57.840708: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-06 15:54:57.860565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 15:54:57.860584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 15:54:57.861114: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 15:54:57.864968: I tensorflow/core/platform/cpu_feature_guar

In [28]:
predictor_tokens = tokens + [' ']

In [29]:
path_to_params = '/home/sl-beast/manikanta/checkpoint/logP/model_params.pkl'
path_to_checkpoint = '/home/sl-beast/manikanta/checkpoint/logP/fold_'

In [30]:
my_predictor = RNNPredictor(path_to_params, path_to_checkpoint, predictor_tokens)

In [32]:
from predictor import VanillaQSAR
from sklearn.ensemble import RandomForestRegressor as RFR

In [34]:
!export CUDA_LAUNCH_BLOCKING=1
!export TORCH_USE_CUDA_DSA=1

In [59]:
smiles_unbiased, prediction_unbiased = estimate_and_update(my_generator,
                                                           my_predictor,
                                                           n_to_generate=100)

Generating molecules...:   0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [30]:
from reinforcement import Reinforcement

In [31]:
my_generator_max = StackAugmentedRNN(input_size=gen_data.n_characters,
                                     hidden_size=hidden_size,
                                     output_size=gen_data.n_characters,
                                     layer_type=layer_type,
                                     n_layers=1, is_bidirectional=False, has_stack=True,
                                     stack_width=stack_width, stack_depth=stack_depth,
                                     use_cuda=use_cuda,
                                     optimizer_instance=optimizer_instance, lr=lr)

my_generator_max.load_model(model_path)

RuntimeError: ignored

In [ ]:
# Setting up some parameters for the experiment
n_to_generate = 200
n_policy_replay = 10
n_policy = 15
n_iterations = 60

In [ ]:
def simple_moving_average(previous_values, new_value, ma_window_size=10):
    value_ma = np.sum(previous_values[-(ma_window_size-1):]) + new_value
    value_ma = value_ma/(len(previous_values[-(ma_window_size-1):]) + 1)
    return value_ma

In [ ]:
def get_reward_logp(smiles, predictor, invalid_reward=0.0):
    mol, prop, nan_smiles = predictor.predict([smiles])
    if len(nan_smiles) == 1:
        return invalid_reward
    if (prop[0] >= 1.0) and (prop[0] <= 4.0):
        return 11.0
    else:
        return 1.0

In [ ]:
x = np.linspace(-5, 12)
reward = lambda x: 11.0 if ((x > 1.0) and (x < 4.0)) else 1.0
plt.plot(x, [reward(i) for i in x])
plt.xlabel('logP value')
plt.ylabel('Reward value')
plt.title('Reward function for logP optimization')
plt.show()

In [ ]:
RL_logp = Reinforcement(my_generator_max, my_predictor, get_reward_logp)

In [ ]:
rewards_max = []
rl_losses_max = []

In [ ]:
for i in range(n_iterations):
    for j in trange(n_policy, desc='Policy gradient...'):
        cur_reward, cur_loss = RL_logp.policy_gradient(gen_data)
        rewards.append(simple_moving_average(rewards, cur_reward))
        rl_losses.append(simple_moving_average(rl_losses, cur_loss))

    plt.plot(rewards)
    plt.xlabel('Training iteration')
    plt.ylabel('Average reward')
    plt.show()
    plt.plot(rl_losses)
    plt.xlabel('Training iteration')
    plt.ylabel('Loss')
    plt.show()

    smiles_cur, prediction_cur = estimate_and_update(RL_logp.generator,
                                                     my_predictor,
                                                     n_to_generate)
    print('Sample trajectories:')
    for sm in smiles_cur[:5]:
        print(sm)

In [ ]:
smiles_biased, prediction_biased = estimate_and_update(RL_logp.generator,
                                                       my_predictor,
                                                       n_to_generate=10000)

In [ ]:
sns.kdeplot(prediction_biased, label='Optimized', shade=True, color='purple')
sns.kdeplot(prediction_unbiased, label='Unbiased', shade=True, color='grey')
plt.xlabel('Predicted logP values')
plt.title('Initial and biased distributions of log P')
plt.legend()
plt.show()

In [ ]:
from rdkit.Chem import Draw

In [ ]:
from rdkit.Chem.Draw import DrawingOptions
from rdkit.Chem import Draw
DrawingOptions.atomLabelFontSize = 50
DrawingOptions.dotsPerAngstrom = 100
DrawingOptions.bondLineWidth = 3

In [ ]:
generated_mols = [Chem.MolFromSmiles(sm, sanitize=True) for sm in smiles_biased]

In [ ]:
sanitized_gen_mols = [generated_mols[i] for i in np.where(np.array(generated_mols) != None)[0]]

In [ ]:
n_to_draw = 20
ind = np.random.randint(0, len(sanitized_gen_mols), n_to_draw)
mols_to_draw = [sanitized_gen_mols[i] for i in ind]
legends = ['log P = ' + str(prediction_biased[i]) for i in ind]

In [ ]:
Draw.MolsToGridImage(mols_to_draw, molsPerRow=5,
                     subImgSize=(200,200), legends=legends)